#Descripción del proyecto.

Telecom X - Análisis de Evasión de Clientes

Has sido contratado como asistente de análisis de datos en Telecom X y formarás parte del proyecto "Churn de Clientes". La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.

Tu desafío será recopilar, procesar y analizar los datos, utilizando Python y sus principales bibliotecas para extraer información valiosa. A partir de tu análisis, el equipo de Data Science podrá avanzar en modelos predictivos y desarrollar estrategias para reducir la evasión.

#Extracción de Datos.


In [3]:
#Importación de librerias.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Exploración inicial.

In [4]:
#Extracción de datos desde un archivo formato Json.

datos = pd.read_json("/content/TelecomX_Data.json")
datos.head(3)

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [6]:
datos.columns

Index(['customerID', 'Churn', 'customer', 'phone', 'internet', 'account'], dtype='object')

#Tratamiento del dataframe.

In [7]:
#Normalizado de tablas
df_customer = pd.json_normalize(datos['customer'])
df_phone = pd.json_normalize(datos['phone'])
df_internet = pd.json_normalize(datos['internet'])
df_account = pd.json_normalize(datos['account'])

In [8]:
#Revisando alguno de los df normalizados para verificar que esta correcto.
df_customer.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4


In [9]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         7267 non-null   object
 1   SeniorCitizen  7267 non-null   int64 
 2   Partner        7267 non-null   object
 3   Dependents     7267 non-null   object
 4   tenure         7267 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 284.0+ KB


In [10]:
#Agregando otra forma para normalizar un df.

ejemplo = datos['customer'].apply(pd.Series)
ejemplo.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4


In [11]:
#Posterior a tener los df normalizados, conecto los df con las columnas en las que no era necesario normalizar. De esta forma tendre un df completo.
# Agregando sobre el eje horizontal
df_telecom = pd.concat([datos[['customerID','Churn']], df_customer, df_phone, df_internet, df_account,], axis=1)
df_telecom.head(3)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [12]:
df_telecom.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [13]:
#Revisando información del df total con el que vamos a trabajar.

df_telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


###Renombre de columnas.

In [14]:
columnas = df_telecom.columns
columnas

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [15]:
#Columnas a minuscula y eliminando el punto que posteriormente daría problemas si no se hace el cambio.

df_telecom.columns = columnas.str.lower().str.strip().str.replace('.', '_')
df_telecom.columns

Index(['customerid', 'churn', 'gender', 'seniorcitizen', 'partner',
       'dependents', 'tenure', 'phoneservice', 'multiplelines',
       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'paperlessbilling', 'paymentmethod', 'charges_monthly',
       'charges_total'],
      dtype='object')

In [16]:
df_telecom.sample(10)

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
3277,4568-KNYWR,No,Male,0,No,No,52,Yes,No,DSL,...,Yes,No,No,Yes,No,Two year,Yes,Credit card (automatic),63.25,3342.45
6362,8746-OQQRW,No,Male,0,No,No,4,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,25.25,101.9
524,0742-NXBGR,Yes,Female,0,No,No,1,Yes,No,Fiber optic,...,Yes,Yes,No,No,No,Month-to-month,Yes,Electronic check,82.30,82.3
1059,1481-ZUWZA,Yes,Male,0,No,No,28,Yes,Yes,Fiber optic,...,Yes,No,No,No,Yes,Month-to-month,Yes,Credit card (automatic),94.50,2659.4
2450,3419-SNJJD,Yes,Female,1,Yes,No,65,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,Yes,Month-to-month,No,Bank transfer (automatic),105.25,6786.4
5130,7008-LZVOZ,No,Male,0,Yes,Yes,66,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),25.30,1672.35
4135,5668-MEISB,No,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),106.10,7657.4
2699,3740-RLMVT,No,Male,1,Yes,No,67,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Bank transfer (automatic),60.95,4119.4
5672,7750-EYXWZ,No,Female,0,No,No,12,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,Yes,One year,No,Electronic check,60.65,743.3
6271,8627-ZYGSZ,No,Male,0,Yes,No,47,Yes,Yes,Fiber optic,...,Yes,No,No,No,No,One year,Yes,Electronic check,78.90,3650.35


#Limpieza de datos.

##Valores nulos

In [17]:
#Según la información de nuestro df, no tenemos valores nulos pero vamos a verificar.

valores_nulos_df_telecom = df_telecom.isnull().sum()
valores_nulos_df_telecom

,0
customerid,0
churn,0
gender,0
seniorcitizen,0
partner,0
dependents,0
tenure,0
phoneservice,0
multiplelines,0
internetservice,0


In [18]:
print(f"A continuación se muestra la cantidad de valores nulos por columna: {valores_nulos_df_telecom}")

A continuación se muestra la cantidad de valores nulos por columna: customerid          0
churn               0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
charges_monthly     0
charges_total       0
dtype: int64


##Valores vacios.

In [19]:
#Para verificar los valores vacios en el df, vamos a utilizar una función lambda
valores_vacios = df_telecom.apply(lambda x: x.astype(str).str.strip() == "").sum()

In [20]:

print(f"En el analisís de valores vacios fueron encontrados en las siguientes dos columnas:")
print(valores_vacios[["churn","charges_total"]])

En el analisís de valores vacios fueron encontrados en las siguientes dos columnas:
churn            224
charges_total     11
dtype: int64


Despues de detectar que en la columna "Churn" y en la columna de "Changes.total" existen valores vacios, solo para ejemplificarlos los resalto a continuación.


###Valores vacios en la columna "Churn".

In [21]:
#Verificando tambien utilizando query de esta forma obtenemos los indices para cada valor vacio en la columna Churn

df_telecom.query("churn == ''").index

Index([  30,   75,   96,   98,  175,  219,  312,  351,  368,  374,
       ...
       7058, 7061, 7062, 7143, 7155, 7158, 7180, 7211, 7239, 7247],
      dtype='int64', length=224)

In [22]:
df_valores_vacios_churn = df_telecom.query("churn == ''")
df_valores_vacios_churn.head()

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45


###Valores vacios en la columna de "changes_total".




In [23]:
#El nombre de la columna "Changes.Total", me da problemas para accesar al indice, me imagino que el punto
#lo interpreta como si estuviese solicitando un metodo o atributo

df_valores_vacios_charges = df_telecom.query("charges_total == ' '")
df_valores_vacios_charges.head()

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,


In [24]:
df_valores_vacios_charges = df_telecom.query("charges_total == ' '")

##Exportando dataframes para posterior estudio.

In [25]:
df_valores_vacios_churn.to_json("Valores_vacios_churn.json")

##Columna "Daily Charges".

In [26]:
#Antes de incluir la nueva columna, voy a cambiar el tipo de dato de "charges_total"

df_telecom['charges_total'] = df_telecom['charges_total'].replace(' ', np.nan).astype(float)


In [27]:
df_telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7267 non-null   object 
 1   churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   seniorcitizen     7267 non-null   int64  
 4   partner           7267 non-null   object 
 5   dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   phoneservice      7267 non-null   object 
 8   multiplelines     7267 non-null   object 
 9   internetservice   7267 non-null   object 
 10  onlinesecurity    7267 non-null   object 
 11  onlinebackup      7267 non-null   object 
 12  deviceprotection  7267 non-null   object 
 13  techsupport       7267 non-null   object 
 14  streamingtv       7267 non-null   object 
 15  streamingmovies   7267 non-null   object 
 16  contract          7267 non-null   object 


In [28]:
#Creando la nueva columna.

df_telecom["daily_charges"] = round(df_telecom['charges_monthly'] / 30, 2)
df_telecom.head(3)

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total,daily_charges
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


In [29]:
df_telecom_copy = df_telecom.copy()


##Metodo UNIQUE

In [30]:
df_telecom_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7267 non-null   object 
 1   churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   seniorcitizen     7267 non-null   int64  
 4   partner           7267 non-null   object 
 5   dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   phoneservice      7267 non-null   object 
 8   multiplelines     7267 non-null   object 
 9   internetservice   7267 non-null   object 
 10  onlinesecurity    7267 non-null   object 
 11  onlinebackup      7267 non-null   object 
 12  deviceprotection  7267 non-null   object 
 13  techsupport       7267 non-null   object 
 14  streamingtv       7267 non-null   object 
 15  streamingmovies   7267 non-null   object 
 16  contract          7267 non-null   object 


In [31]:
valores_uniq = df_telecom_copy.apply(lambda x: x.unique())
valores_uniq

,0
customerid,"[0002-ORFBO, 0003-MKNFE, 0004-TLHLJ, 0011-IGKF..."
churn,"[No, Yes, ]"
gender,"[Female, Male]"
seniorcitizen,"[0, 1]"
partner,"[Yes, No]"
dependents,"[Yes, No]"
tenure,"[9, 4, 13, 3, 71, 63, 7, 65, 54, 72, 5, 56, 34..."
phoneservice,"[Yes, No]"
multiplelines,"[No, Yes, No phone service]"
internetservice,"[DSL, Fiber optic, No]"


##Estandarización de columna "churn".

In [32]:
#Cambiando los valores "YES" y "NO" a valores númericos. Como lo dicta la columna "abandono"
#NO corresponde a los clientes que aun mantienen el servicio. --> 0
#YES corresponde a los clientes que abandonaron el servicio. --> 1

df_telecom_copy['churn'] = df_telecom_copy['churn'].map({"No": 0, "Yes":1})

##Eliminando celdas con valores nulos.



In [33]:
df_telecom_copy.dropna(subset=["churn"], inplace = True)

In [34]:
df_telecom_copy['churn'] = df_telecom_copy['churn'].astype(int)

In [35]:
df_telecom_copy.head(3)

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total,daily_charges
0,0002-ORFBO,0,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,0,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,1,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


##Estandarización de columnas a valores númericos.

In [36]:
columnas_numericas = ["onlinesecurity",
                      "onlinebackup",
                      "deviceprotection",
                      "techsupport",
                      "streamingtv",
                      "streamingmovies"]



In [37]:
mapeo = {"No": 0, "Yes": 1, "No internet service": -1}

In [38]:
"""for col in columnas_numericas:
    df_telecom_copy[col] = df_telecom_copy[col].map({"No": 0, "Yes": 1, "No internet service": -1})"""

'for col in columnas_numericas:\n    df_telecom_copy[col] = df_telecom_copy[col].map({"No": 0, "Yes": 1, "No internet service": -1})'

In [39]:
#Este metodo funciona pero me salta un warning que puede dar algún problema para futuras versiones de pandas.
df_telecom_copy[columnas_numericas] = df_telecom[columnas_numericas].applymap(lambda x: {"No": 0, "Yes": 1, "No internet service": -1}.get(x, x))

/tmp/ipython-input-39-1272472340.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_telecom_copy[columnas_numericas] = df_telecom[columnas_numericas].applymap(lambda x: {"No": 0, "Yes": 1, "No internet service": -1}.get(x, x))


In [40]:
df_telecom_copy.sample(10)

,customerid,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total,daily_charges
6550,8999-EXMNO,0,Female,0,Yes,Yes,19,Yes,No,No,...,-1,-1,-1,-1,Two year,No,Mailed check,20.15,387.70,0.67
6554,9003-CPATH,0,Male,0,No,No,42,Yes,Yes,Fiber optic,...,1,0,0,0,Month-to-month,No,Electronic check,79.90,3313.40,2.66
2534,3537-HPKQT,0,Female,0,Yes,No,55,Yes,Yes,Fiber optic,...,0,0,0,0,Month-to-month,Yes,Credit card (automatic),77.75,4266.40,2.59
1592,2263-SFSQZ,0,Male,0,Yes,Yes,51,Yes,Yes,No,...,-1,-1,-1,-1,Two year,Yes,Mailed check,24.95,1222.25,0.83
1193,1702-CCFNJ,0,Male,0,Yes,No,52,Yes,Yes,DSL,...,0,1,0,0,Month-to-month,No,Bank transfer (automatic),61.35,3169.55,2.04
1232,1763-KUAAW,0,Female,1,No,No,18,Yes,No,No,...,-1,-1,-1,-1,One year,No,Bank transfer (automatic),20.35,369.60,0.68
4286,5893-PYOLZ,0,Male,0,No,No,56,Yes,No,DSL,...,1,1,1,1,Two year,Yes,Credit card (automatic),81.80,4534.45,2.73
4118,5649-RXQTV,0,Male,0,No,No,51,Yes,No,Fiber optic,...,1,0,1,1,Month-to-month,Yes,Electronic check,99.00,5038.15,3.30
116,0196-VULGZ,1,Female,1,Yes,No,6,Yes,Yes,Fiber optic,...,0,0,0,1,Month-to-month,Yes,Electronic check,84.35,474.90,2.81
2803,3887-PBQAO,0,Female,0,Yes,Yes,45,Yes,Yes,No,...,-1,-1,-1,-1,One year,Yes,Credit card (automatic),25.90,1216.60,0.86


##Análisis descriptivo.

In [41]:
columnas_analisis = ["seniorcitizen","tenure","charges_monthly","charges_total","daily_charges"]

In [42]:
df_describe = round(df_telecom_copy[columnas_analisis].describe(), 2)

###Dataframe de análisis descriptivo.

In [43]:
df_describe

,seniorcitizen,tenure,charges_monthly,charges_total,daily_charges
count,7043.00,7043.00,7043.00,7032.00,7043.00
mean,0.16,32.37,64.76,2283.30,2.16
std,0.37,24.56,30.09,2266.77,1.00
min,0.00,0.00,18.25,18.80,0.61
25%,0.00,9.00,35.50,401.45,1.18
50%,0.00,29.00,70.35,1397.48,2.34
75%,0.00,55.00,89.85,3794.74,2.99
max,1.00,72.00,118.75,8684.80,3.96


##Distribución de evasión.

In [44]:
import plotly.express as px

###Distribución de clientes, genero-abandono.

In [57]:
fig_genero = px.histogram(
    df_telecom,
    x = "gender",
    text_auto = True,
    color = "churn",
    barmode = "group",
    title= "Distribución de clientes por genero y abandono",
    color_discrete_map={"Yes": "crimson", "No":"steelblue"})
fig_genero.update_traces(textposition="outside")
fig_genero.update_layout(
    title_font_size=22,
    title_x = 0.5,
    xaxis_title ="Género",
    yaxis_title ="Número de Clientes",
    legend_title="Abandono",
    plot_bgcolor="white",
    bargap=0.3)
fig_genero.update_layout(
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=1,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=1,
    )
)

fig_genero.update_layout(
    width=900,
    height=450
)
fig_genero.show()

In [46]:
print(dir(px))

['Constant', 'IdentityMap', 'NO_COLOR', 'Range', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_chart_types', '_core', '_doc', '_imshow', '_special_inputs', 'area', 'bar', 'bar_polar', 'box', 'choropleth', 'choropleth_map', 'choropleth_mapbox', 'colors', 'data', 'defaults', 'density_contour', 'density_heatmap', 'density_map', 'density_mapbox', 'ecdf', 'funnel', 'funnel_area', 'get_trendline_results', 'histogram', 'icicle', 'imshow', 'imshow_utils', 'line', 'line_3d', 'line_geo', 'line_map', 'line_mapbox', 'line_polar', 'line_ternary', 'optional_imports', 'parallel_categories', 'parallel_coordinates', 'pd', 'pie', 'scatter', 'scatter_3d', 'scatter_geo', 'scatter_map', 'scatter_mapbox', 'scatter_matrix', 'scatter_polar', 'scatter_ternary', 'set_mapbox_access_token', 'strip', 'sunburst', 'timeline', 'treemap', 'trendline_functions', 'violin']


###Distribución de clientes, tipo de contrato-abandono.

In [59]:
fig_tipocontrato = px.histogram(
    df_telecom,
    x = "contract",
    text_auto = True,
    color = "churn",
    barmode = "group",
    title= "Distribución de clientes por tipo de contrato y abandono",
    color_discrete_map={"Yes": "crimson", "No":"steelblue"})
fig_tipocontrato.update_traces(textposition="outside")
fig_tipocontrato.update_layout(
    title_font_size=22,
    title_x = 0.5,
    xaxis_title ="Tipo de contrato",
    yaxis_title ="Número de Clientes",
    legend_title="Abandono",
    plot_bgcolor="white",
    bargap=0.3)
fig_tipocontrato.update_layout(
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    )
)

fig_tipocontrato.update_layout(
    width=900,
    height=500
)
fig_tipocontrato.show()

In [48]:
df_telecom.columns

Index(['customerid', 'churn', 'gender', 'seniorcitizen', 'partner',
       'dependents', 'tenure', 'phoneservice', 'multiplelines',
       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'paperlessbilling', 'paymentmethod', 'charges_monthly', 'charges_total',
       'daily_charges'],
      dtype='object')

In [60]:
fig_tipopago = px.histogram(
    df_telecom,
    x = "paymentmethod",
    text_auto = True,
    color = "churn",
    barmode = "group",
    title= "Distribución de clientes por tipo de pago y abandono",
    color_discrete_map={"Yes": "crimson", "No":"steelblue"})
fig_tipopago.update_traces(textposition="outside")
fig_tipopago.update_layout(
    title_font_size=22,
    title_x = 0.5,
    xaxis_title ="Tipo de pago",
    yaxis_title ="Número de Clientes",
    legend_title="Abandono",
    plot_bgcolor="white",
    bargap=0.3)
fig_tipopago.update_layout(
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    )
)

fig_tipopago.update_layout(
    width=900,
    height=500
)
fig_tipopago.show()

###Distribución de clientes, permanencia-abandono.

In [50]:
#Mantengo el histograma solo por practica.
fig = px.histogram(
    df_telecom,
    x = "tenure",
    text_auto = True,
    color = "churn",
    barmode = "group",
    title= "Distribución de clientes por tipo de pago y abandono",
    color_discrete_map={"Yes": "crimson", "No":"steelblue"})
fig.update_traces(textposition="outside")
fig.update_layout(
    title_font_size=22,
    title_x = 0.5,
    xaxis_title ="Tipo de pago",
    yaxis_title ="Número de Clientes",
    legend_title="Abandono",
    plot_bgcolor="white",
    bargap=0.3)
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    )
)

fig.update_layout(
    width=900,
    height=500
)
fig.show()

In [51]:
figboxplot = px.box(
    df_telecom,
    x = "tenure",
    color = "churn",
    title= "Distribución de clientes, abandono-pertenencia",
    color_discrete_map={"Yes": "crimson", "No":"steelblue"})

figboxplot.update_layout(
    title_font_size=22,
    xaxis_title ="meses",
    yaxis_title ="Número de Clientes",
    legend_title="Abandono",
    plot_bgcolor="white",)
figboxplot.update_layout(
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        gridwidth=0.3,
    )
)

fig.update_layout(
    width=900,
    height=500
)
fig.show()

In [52]:
#Porcentaje de abandono en relación a tenure"
df_agrupado = df_telecom.groupby(["tenure", "churn"]).size().unstack(fill_value=0)

# % de churn por tenure
df_agrupado["churn_rate"] = round((df_agrupado.get("Yes", 0) / df_agrupado.sum(axis=1)) * 100, 2)

print(df_agrupado["churn_rate"])

tenure
0      0.00
1     59.94
2     50.00
3     45.41
4     44.86
      ...  
68     8.65
69     8.00
70     9.02
71     3.41
72     1.63
Name: churn_rate, Length: 73, dtype: float64


In [53]:
df_telecom.columns

Index(['customerid', 'churn', 'gender', 'seniorcitizen', 'partner',
       'dependents', 'tenure', 'phoneservice', 'multiplelines',
       'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'paperlessbilling', 'paymentmethod', 'charges_monthly', 'charges_total',
       'daily_charges'],
      dtype='object')

#Informe Final




###Introducción


El presente análisis tiene como objetivo explorar el fenómeno de evasión de clientes o "churn", es decir, aquellos usuarios que han decidido cancelar su servicio. Comprender las causas y patrones asociados a la cancelación es crucial para desarrollar estrategias de retención efectivas y reducir la perdida de clientes.

###Limpieza y tratamiento de datos.

Para el tratamiento de los datos se utilizo principalmente la librería Pandas. Originalmente se presenta en formato .json. Este formato tiene a tener columnas anidadas por lo que se tuvo que normalizar para obtener un dataframe completo. Posterior a la normalización se realizo una exploracion general para determinar si este contaba con valores nulos o valores vacios y finalmente realizar una reestructuración del dataset para mayor legibilidad.

###Análisis exploratorio de datos.

In [55]:
df_describe

,seniorcitizen,tenure,charges_monthly,charges_total,daily_charges
count,7043.00,7043.00,7043.00,7032.00,7043.00
mean,0.16,32.37,64.76,2283.30,2.16
std,0.37,24.56,30.09,2266.77,1.00
min,0.00,0.00,18.25,18.80,0.61
25%,0.00,9.00,35.50,401.45,1.18
50%,0.00,29.00,70.35,1397.48,2.34
75%,0.00,55.00,89.85,3794.74,2.99
max,1.00,72.00,118.75,8684.80,3.96


Si bien el análisis fue realizado en la mayoria de las categorias del dataframe. A continuación se muestran algunas visualizaciones solo de aquellas categorias que cuentan con más patrones.

In [61]:
fig_tipopago

In [62]:
fig_tipocontrato

In [63]:
fig.show()

###Insights

##Recomendaciones.

Tomando en cuenta las visualizaciones, podemos poner atención en 3 categorias.

* Muchos clientes llevan poco tiempo, al menos el 25% tiene menos de 9 meses. Lo que nos indica que los clientes nuevos tienden a abandonar más. Estrategia: Se pudiesen mejorar los planes iniciales, incluso agregar algun tipo de promoción que incluya por ejemplo algun servicio de streaming gratis con el primer año de contrato, esto para mejorar la retención.

* Según el gráfico de "Tipo de contrato", el contrato con mas abandono es para el pago, "mes con mes". Como estrategia se sugiere incentivar a los clientes a cambiar a algun otro tipo de contrato, un contrato anual o idealmente un contrato de dos años agregando algun beneficio por su contratación, regalando meses gratis o algun servicio de streaming.

* Para "tipo de pago", se aprecia que los clientes que hacen el pago de su servicio con un "electronic check" tienden a cancelar mas su servicio. Como estrategia se sugiere eliminar la categoria, de igual forma se puede incentivar al cliente para actualizar el tipo de pago a uno mas automatico, de esta forma se puede asegurar la retención.